In [1]:
import numpy as np
import pandas as pd
import nltk
import string
import random
import json
import re
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
with open('/content/drive/MyDrive/conversation.json', 'r', errors='ignore') as f:
    data = json.load(f)

# Convert data to DataFrame
df = pd.DataFrame(data['intents'])
dress_designs = data.get('dress_designs', {})
measurements_data = data.get('measurements_data', {})

In [4]:
patterns = []
responses = []
tags = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        responses.append(intent['responses'])
        tags.append(intent['tag'])

In [5]:
raw_doc = ' '.join(patterns)  # Combine all patterns into a single document
raw_doc = raw_doc.lower()  # Convert text to lowercase
nltk.download('punkt')  # Download the Punkt tokenizer
nltk.download('wordnet')  # Download the WordNet dictionary

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
lemmer = nltk.stem.WordNetLemmatizer()

def LemNormalize(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    return [lemmer.lemmatize(token) for token in nltk.word_tokenize(text.lower().translate(remove_punct_dict))]

In [7]:
TfidVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
tfidf = TfidVec.fit_transform(patterns)

# Define greetings and farewells
greetings = ["hi", "hello", "hey", "good morning", "good afternoon", "good evening"]
farewells = ["bye", "goodbye", "see you later", "take care"]

# Initialize conversation context
context = {'prev_input': None, 'prev_response': None}

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [8]:
def get_previous_line(file_path, target_line="Give the design ID you've chosen"):
    previous_line = None
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for i, line in enumerate(lines):
            if line.strip() == target_line:
                # Check if it's not the first line
                if i > 0:
                    previous_line = lines[i - 1].strip()
                break
    return previous_line

In [9]:
def get_tailor_details(tailor_id, data):
    tailors = data.get('tailors', [])
    for tailor in tailors:
        if tailor['tailor_id'] == tailor_id:
            return tailor
    return None

In [10]:
def generate_response(user_input, context):
    global tfidf, responses, tags

    # Check if user input is a greeting or farewell
    if user_input.lower() in greetings:
        return random.choice(responses[tags.index("greeting")]), context
    elif user_input.lower() in farewells:
        return random.choice(responses[tags.index("farewell")]), context

    # Check if user input is a number (potential design ID)
    if user_input.isdigit():
        design_id = user_input
        # Check if the design ID exists in the dress designs
        if design_id in dress_designs:
            design_details = dress_designs[design_id]
            context['chosen_design'] = design_details
            context['design_id'] = design_id
            response = f"You have chosen the following design:\nStyle: {design_details['style']}\nPlease enter the color."
            return response, context
        else:
            response = "I'm sorry, I couldn't find the design with that ID. Please enter a valid design ID."
            return response, context

    # Check if the user has chosen a design and prompt for color and fabric
    if 'chosen_design' in context:
        chosen_design = context['chosen_design']
        design_id = context['design_id']

        if 'color' not in context:
            context['color'] = user_input
            response = f"You have chosen the color: {user_input}.\nPlease enter the fabric."
            return response, context
        elif 'fabric' not in context:
            context['fabric'] = user_input
            design_details = context['chosen_design']  # Fetch design details from context
            response = f"You have chosen the fabric: {user_input}\nConfirm your selection of\ndesign: {design_details['style']}\ncolor: {context['color']}\nfabric: {context['fabric']}? (Yes/No)"
            return response, context

    if 'chosen_design' in context and user_input.lower() == 'yes':

      chosen_dress_style = get_previous_line('history.txt')
      for dress_style_data in measurements_data:
          if dress_style_data["dress_style"] == chosen_dress_style.lower():
              dress_style = dress_style_data["dress_style"]
              print(f"\nDress Type: {dress_style}")
              print("Please select your measurements:")

              sizingOptions = dict(list(dress_style_data.items())[1:])
              for category, options in sizingOptions.items():
                  print(f"\n{category}:")
                  for option in options:
                      print(option)
                  selected_option = input("Enter your choice: ").strip().upper()
                  while not any(selected_option.startswith(opt.split('(')[0].strip().upper()) for opt in options):
                      print("Invalid choice. Please select again.")
                      selected_option = input("Enter your choice: ").strip().upper()
                  dress_style_data[category] = selected_option
              print("\nSelected Measurements:")
              print(json.dumps(dress_style_data, indent=2))
              with open('measurements.json', 'a') as f:
                  json.dump(dress_style_data, f, indent=2)
              dress_style_data = ''
              break
      response = "Measurements selected successfully."
      tailor_id = input("Please enter the tailor ID you want to connect to from the website: ")
      selected_tailor = get_tailor_details(tailor_id, data)
      if selected_tailor:
        response = f"Great! You've chosen tailor {selected_tailor['name']} located in {selected_tailor['location']}. Contact them at {selected_tailor['contact']} for further assistance."

        '''# Store customer's information
        customer_information = {
            'measurements': context.get('chosen_design', {}),
            'history': context.get('prev_input', '')
        }
        # Connect the customer and the tailor
        print("Customer Information:")
        print(json.dumps(customer_information, indent=2))'''
        print("Tailor Details:")
        print(json.dumps(selected_tailor, indent=2))
      else:
        response = "Invalid tailor ID. Please enter a valid ID."
      return response, context

    elif user_input.lower() == 'no':
        if 'fabric' in context:
            del context['fabric']  # Remove fabric from context if already entered
        if 'color' in context:
            del context['color']  # Remove color from context if already entered
        del context['chosen_design']  # Reset chosen design
        response = "Conversation has been rolled back! Please enter the design ID."

        # Remove the last two lines from the history file
        with open('history.txt', 'r+') as f:
            lines = f.readlines()
            f.truncate(0)  # Clear the file
            f.seek(0)
            f.writelines(lines[:-2])  # Write all lines except the last two

        return response, context

    elif re.match(r'\breset\b', user_input.lower()):
        context.clear()  # Reset context
        response = "Conversation has been reset. Please start again."

        # Clear the history file
        open('history.txt', 'w').close()
        open('measurements.json', 'w').close()

        return response, context

    # Vectorize the user input
    user_tfidf = TfidVec.transform([user_input])

    # Compute cosine similarity between user input and sentences
    vals = cosine_similarity(user_tfidf, tfidf)
    idx = vals.argsort()[0][-1]
    req_tfidf = vals[0][idx]

    if req_tfidf == 0:
        response = "I am sorry! I don't understand you."
    else:
        response = responses[idx][0]

    # Update context with current input and response
    context['prev_input'] = user_input
    context['prev_response'] = response

    return response, context

################ MAIN BOT RUNNER ###################

# Chat loop
print("Bot: I'm Stark. Feel free to ask me anything")

# History File
with open('history.txt', 'w') as f:
    f.seek(0)
    f.truncate()

# Measurements File
with open('measurements.json', 'w') as f:
    f.seek(0)
    f.truncate()

context = {}

while True:
    user_input = input("User: ")

    # Terminate conversation for certain inputs
    if re.match(r'.*(hate|kill|die).*', user_input.lower()):
        print("Bot: I'm sorry, I cannot engage in such conversations. Goodbye!")
        break

    if 'bye' in user_input.lower():
        print("Bot: Goodbye! Take care.")
        break

    # Generate response based on user input and context
    response, context = generate_response(user_input, context)

    print("Bot:", response)

    # Update context
    context['prev_input'] = user_input
    context['prev_response'] = response
    with open('history.txt', 'a+') as f:
        f.write(f'{user_input}\n{response}\n')

Bot: I'm Stark. Feel free to ask me anything
User: Hi
Bot: Hello there. Customize stitching services at your doorstep! What dress style are you looking for?
User: Kurti
Bot: Give the design ID you've chosen
User: 20
Bot: You have chosen the following design:
Style: Herringbone
Please enter the color.
User: Black
Bot: You have chosen the color: Black.
Please enter the fabric.
User: Cotton
Bot: You have chosen the fabric: Cotton
Confirm your selection of
design: Herringbone
color: Black
fabric: Cotton? (Yes/No)
User: Yes

Dress Type: kurti
Please select your measurements:

Chest:
S(38)
M(40)
L(42)
XL(44)
XXL(46)
XXXL(48)
Enter your choice: M

Waist:
S(34)
M(36)
L(38)
XL(40)
XXL(42)
XXXL(44)
Enter your choice: S

Hip:
S(39)
M(41)
L(43)
XL(45)
XXL(47)
XXXL(49)
Enter your choice: S

Short tunic length:
S(26-34)
M(26-34)
L(26-34)
XL(26-34)
XXL(26-34)
XXXL(26-34)
Enter your choice: M

Long kurti length:
S(38/42)
M(38/42)
L(38/42)
XL(38/42)
XXL(38/42)
XXXL(38/42)
Enter your choice: M

Sleeve l